<a href="https://colab.research.google.com/github/trentfridey/qosf-screening/blob/master/task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pennylane

     |████████████████████████████████| 317kB 2.9MB/s 


In [11]:
import pennylane as qml
from pennylane import numpy as np
import torch
from torch.autograd import Variable

n_wires = 2

target = torch.tensor([0, 0.5, 0.5, 0])

dev = qml.device('default.qubit', wires=2)

def ansatz(params):
  qml.RX(params[0], wires=0)
  qml.RY(params[1], wires=1)
  qml.CNOT(wires=[0,1])

@qml.qnode(dev, interface="torch")
def circuit(params):
  ansatz(params)
  return qml.probs(wires=[0,1])

def cost(params):
  cost = torch.sum(torch.abs(circuit(params) - target))
  return cost

In [25]:
# Optimization

# initialize params
params = np.random.normal(0, np.pi, 2)
params = Variable(torch.tensor(params), requires_grad=True)

# number of steps
steps = 200
opt = torch.optim.Adam([params], lr=0.1)

# keeping track along the way
best_cost = cost(params)
best_params = np.zeros(2)

print("Initial cost: {}".format(cost(params)))

for n in range(steps):
  opt.zero_grad()
  loss = cost(params)
  loss.backward()
  opt.step()

  if loss < best_cost:
      best_cost = loss
      best_params = params

  if n % 10 == 9 or n == steps - 1:
      print("Cost after {} steps is {:.4f}".format(n+1, loss))

print("The best gates are Rx({}) and Ry({})".format(best_params[0].item(), best_params[1].item()))

Initial cost: 1.0209467233548617
Cost after 10 steps is 0.6500
Cost after 20 steps is 0.1968
Cost after 30 steps is 0.0367
Cost after 40 steps is 0.0187
Cost after 50 steps is 0.0019
Cost after 60 steps is 0.0100
Cost after 70 steps is 0.0093
Cost after 80 steps is 0.0018
Cost after 90 steps is 0.0078
Cost after 100 steps is 0.0091
Cost after 110 steps is 0.0016
Cost after 120 steps is 0.0074
Cost after 130 steps is 0.0088
Cost after 140 steps is 0.0015
Cost after 150 steps is 0.0072
Cost after 160 steps is 0.0086
Cost after 170 steps is 0.0015
Cost after 180 steps is 0.0071
Cost after 190 steps is 0.0085
Cost after 200 steps is 0.0015
The best gates are Rx(4.712808692667411) and Ry(-3.1416110352548867)
